In [ ]:
import pinecone
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm.auto import tqdm
import re

In [ ]:

# Initialize Pinecone
pinecone.init(api_key="<YOUR-API-KEY>", environment="<YOUR-ENVIRONMENT>")


In [ ]:

# Initialize the tokenizer and model (OpenAI's GPT-2 model is used as an example)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = AutoModel.from_pretrained("gpt2")


In [ ]:
df=pd.read_csv('covid19_tweets.csv')

In [ ]:
df['text']=df['text'].str.replace(r'http\S+', '', flags=re.MULTILINE)

In [ ]:
df['text'] = df['text'].str.replace(r'[^\w\s]', ' ')

In [ ]:
def remove_usernames_links(tweet):
        # tweet = re.sub('@[^\s]+','',str(tweet))
        # tweet = re.sub('#[^\s]+','',str(tweet))
        tweet = re.sub('http[^\s]+','',str(tweet))
        return tweet
df['text'] = df['text'].apply(remove_usernames_links)

In [ ]:
df=df[:5000]

In [ ]:
df.head()

In [ ]:
# A function to encode text into vector
def encode_text(text):
    global i
    # inputs = tokenizer(text, return_tensors="pt")
    # if len(inputs) > 512:
    #     print(text)
    #     inputs = inputs[:512]
    # outputs = model(**inputs)
    # return outputs.last_hidden_state.mean(dim=1).detach().numpy()
    # print(text)
    inputs = tokenizer.encode(text, return_tensors="pt")
    # print(inputs)
    # print(len(inputs[0]))
    if len(inputs[0]) > 512:
        # print(text)
        inputs = inputs[0][:512]
    with torch.no_grad():
        embeddings = model(inputs)[0]
    # print(embeddings)
    # if len(embeddings[0]) > 512:
    #     # print(embeddings[0], len(embeddings[0]))
    #     embeddings[0] = embeddings[0][:512]
    # print(len(embeddings[0]))
    # print(i,end="")
    # i+=1
    return embeddings.mean(dim=1).squeeze().numpy()


In [ ]:
# i=1
# Assuming 'df' is your DataFrame and it has a column 'text' which contains the text documents
vectors = df['text'].apply(encode_text)


In [ ]:
vectors = vectors.tolist()

In [ ]:
len(vectors[14])

In [ ]:
# df['abstract'][13]

In [ ]:
ls=[]
vs = []
for i in range(len(vectors)):
    if len(vectors[i].tolist()) == 768:
        vectors[i][0] = vectors[i][0].astype('float64')
        vs.append(vectors[i].tolist())
        ls.append({"id":str(i),"values":vectors[i].tolist()})
    # ls.append(vectors[i][0])

In [ ]:
print(len(ls))

In [ ]:
for i in ls:
    if len(i['values']) != 768:
        print(i['id'], len(i['values']))
        

In [ ]:
# vs

In [ ]:

# Create a Pinecone index
if "semantic-search" in pinecone.list_indexes():
    pinecone.delete_index("semantic-search")
pinecone.create_index(name="semantic-search",dimension=768,metric='cosine')


In [ ]:
index = pinecone.Index("semantic-search")

In [ ]:

# Upsert vectors into Pinecone
batch_size = 100

for i in tqdm(range(0, len(df), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(df))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [{'text': df['text'].iloc[i]} for i in range(i,i_end)]
    print(metadatas)
    # create embeddings
    xc = vs[i:i_end]
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

# check number of records in the index
index.describe_index_stats()

# index.upsert(vectors=ls)


In [ ]:

# Function for semantic search
def search(query):
    query_vector = encode_text(query)
    query_vector = query_vector.astype('float64')
    query_vector = query_vector.tolist()
    
    results = index.query(vector=query_vector, top_k=10)
    return results


In [ ]:
# i=1

In [ ]:

# Search
query = "eat healthy food during this pandamic period"
res = search(query)
print(res)
for j in res['matches']:
    # print(df.iloc[int(j['id'])]['title'],"\n")
    print(df.iloc[int(j['id'])]['text'])
    print("Similarity: ",j['score'],"\n\n")

In [ ]:
df.to_csv('covid19_tweets_new.csv',index=False)